In [1]:
%matplotlib nbagg

In [2]:
import numpy as np
import distributed as dd
import matplotlib.pyplot as plt

from libertem import api
from libertem.udf.stddev import StdDevUDF
from libertem.udf.sum import SumUDF
from libertem.udf.raw import PickUDF
from libertem.contrib.daskadapter import make_dask_array

In [3]:
ctx = api.Context()

In [4]:
ds = ctx.load("auto", path='/cachedata/users/clausen/libertem-test-data/Capture52/Capture52_.gtg')

In [5]:
roi = np.ones(ds.shape.nav, dtype=bool)

In [6]:
stddev_1 = ctx.run_udf(udf=StdDevUDF(), dataset=ds)

In [7]:
stddev_2 = ctx.run_udf(udf=StdDevUDF(), dataset=ds, roi=roi)

In [8]:
np.allclose(stddev_1['varsum'].data, stddev_2['varsum'].data)

True

In [9]:
np.allclose(stddev_1['sum'].data, stddev_2['sum'].data)

True

In [10]:
stddev_1['num_frames'].data - stddev_2['num_frames'].data

array([0])

In [11]:
sum_1 = ctx.run_udf(udf=SumUDF(), dataset=ds)

In [12]:
sum_2 = ctx.run_udf(udf=SumUDF(), dataset=ds, roi=roi)

In [13]:
np.allclose(sum_1['intensity'].data, sum_2['intensity'].data)

True

In [14]:
np.allclose(sum_1['intensity'].data, stddev_2['sum'].data)

True

In [15]:
da, workers = make_dask_array(ds, dtype=np.float64)

In [16]:
res = ctx.executor.client.compute(((da.mean(axis=(0, 1)) - da)**2).sum(axis=(0,1)))

In [17]:
np.allclose(res.result(), stddev_1['varsum'].data)

True

In [18]:
res_sum = ctx.executor.client.compute(da.sum(axis=(0, 1)))

In [19]:
np.allclose(res_sum.result(), sum_1['intensity'].data)

True

In [20]:
np.max(res_sum.result() - sum_1['intensity'].data)

0.0